In [1]:
import pandas as pd
import numpy as np
import os

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# Random Forest
결정 트리를 랜덤하게 만들어 결정트리의 숲을 만든다  
입력한 트레인 셋에서 랜덤하게 샘플을 추출해 트레인 셋을 만든다. 단, 샘플이 중복되어 추출될 수 있다      
criterion, max_depth, max_features, min_samples_split, min_impurity_decrease, min_samples_leaf 등의 `DecisionTreeClassifier`가 제공하는 주요 매개변수를 모두 제공한다.


  
- `RandomForestClassifier`는 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택한다 (4개의 특성이 있다면, 노드마다 2개를 랜덤선택해 사용한다)   
- `RandomForestRegressor`는 전체 특성을 사용한다

In [2]:
df = pd.read_csv('https://bit.ly/wine-date')
data = df[['alcohol', 'sugar', 'pH']].to_numpy()
target = df['class'].to_numpy()

train_input, test_input, train_target, test_target = \
    train_test_split(data, target, test_size=0.2, random_state = 42)

In [3]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

In [4]:
rf = RandomForestClassifier(n_jobs= -1, random_state=42)

In [5]:
scores = cross_validate(rf, train_input, train_target, \
                       return_train_score=True, n_jobs = -1)

In [6]:
scores

{'fit_time': array([0.290205  , 0.28714561, 0.22601151, 0.28962922, 0.32804418]),
 'score_time': array([0.02370834, 0.02383828, 0.03880858, 0.03324461, 0.03239155]),
 'test_score': array([0.88461538, 0.88942308, 0.90279115, 0.88931665, 0.88642926]),
 'train_score': array([0.9971133 , 0.99663219, 0.9978355 , 0.9973545 , 0.9978355 ])}

In [7]:
np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.9973541965122431, 0.8905151032797809)

In [8]:
rf.fit(train_input, train_target)
rf.feature_importances_

# 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻는다.
# 이는 과대적합을 줄이고 일반화 성능을 높히는데 도움을 준다.

array([0.23167441, 0.50039841, 0.26792718])

In [9]:
rf = RandomForestClassifier(oob_score=True, n_jobs = -1, random_state=42)
rf.fit(train_input, train_target)

RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=42)

In [10]:
# 부트스트랩 샘플에 포함되지 않고 남는 샘플 > Out Of Bag
# 이를 이용해 부트스트랩 샘플로 훈련한 결정트리를 평가한다. 
# 검증 세트의 역할을 하는 셈

rf.oob_score_

0.8934000384837406

## 엑스트라 트리
부트스트랩 샘플을 사용하지 않는다. 즉, 각 결정트리를 만들 때 전체 트레인 셋을 사용한다  
대신 노드를 분할할 때 가장 좋은 분할을 찾지 않고, **무작위로 분할** 한다.   

엑스트라 트리가 무작위성이 더 크기 때문에 랜덤 포레스트보다 더 많은 결정트리를 훈련한다.  
하지만 랜덤하게 노드를 분할하기 때문에 빠른 계산속도를 가진다.  

`DecisionTreeClassifier(splitter='random')`

In [12]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs = -1, random_state= 42)
scores = cross_validate(et, train_input, train_target,
                return_train_score=True, n_jobs = -1)

np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.9974503966084433, 0.8887848893166506)

In [13]:
et.fit(train_input, train_target)
et.feature_importances_

array([0.20183568, 0.52242907, 0.27573525])

# Gradient Boosting
깊이가 얕은 결정 트리를 사용해 이전 트리의 오차를 보완한다. 과대 적합에 강하고, 일반적으로 높은 일반화 성능을 기대할 수 있다.   
분류에서는 로지스틱 손실함수를 사용하고, 회귀에서는 평균 제곱 오차 함수 MSE를 사용한다.

In [14]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target,
                return_train_score=True, n_jobs=-1)

np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.8881086892152563, 0.8720430147331015)

In [15]:
gb = GradientBoostingClassifier(n_estimators = 500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target,
                return_train_score=True, n_jobs=-1)

np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.9464595437171814, 0.8780082549788999)

In [16]:
gb.fit(train_input, train_target)
gb.feature_importances_

array([0.15872278, 0.68010884, 0.16116839])

`subsample` : 트리 훈련에 사용할 트레인 셋의 비율을 정함   
`subsample`이 1보다 작으면 트레인 셋의 일부를 사용한다. 이는 경사 하강법 단계마다 일부 샘플을 랜덤하게 선택해 진행하는 확률적 경사 하강법이나 미니배치 경사 하강법과 비슷하다.

# Histogram-based Gradient Boosting
히스토그램 기반 그레디언트 부스팅  



입력 특성을 256개의 구간으로 나눈다. 따라서 노드를 분할할 때 최적의 분할을 매우 빠르게 찾는다.  
256개의 구간 중 하나를 떼어 놓고 누락된 값을 위해 사용한다. 따라서 입력에 누락된 특성이 있더라도 따로 전처리 할 필요가 없다.  

사이킷런에서 `HistGradientBoostingClassifier` 는 트리 개수 지정에서 `m_estimators` 대신 부스팅 반복 횟수를 지정하는 `max_iter`를 사용한다.

In [17]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

In [20]:
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

# 과대적합을 억제하면서 Gradient boosting 보다 높은 성능을 제공한다.
np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.9321723946453317, 0.8801241948619236)

In [21]:
hgb.fit(train_input, train_target)

HistGradientBoostingClassifier(random_state=42)

In [24]:
rf.feature_importances_

array([0.23167441, 0.50039841, 0.26792718])

In [25]:
hgb.feature_importances_

AttributeError: 'HistGradientBoostingClassifier' object has no attribute 'feature_importances_'

In [27]:
from xgboost import XGBClassifier

In [28]:
xgb = XGBClassifier(tree_method = 'hist', random_state = 42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)

np.mean(scores['train_score']), np.mean(scores['test_score'])

/home/eunbinpark/miniconda3/envs/da/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:23:20] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:23:20] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:23:20] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:23:20] WARNING: /home/conda/feedstock_r

(0.9555033709953124, 0.8799326275264677)

In [29]:
from lightgbm import LGBMClassifier

In [30]:
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

In [31]:
np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.935828414851749, 0.8801251203079884)